# Ceci est un guide étape par étape pour gratter les tweets Twitter à l'aide d'une bibliothèque Python appelée Tweepy.
### Étude de cas: Covid 2019 vaccin 2019


Prérequis: créer un compte de développeur Twitter
Avant de commencer à utiliser Tweepy, vous aurez besoin d'un compte de développeur Twitter pour appeler les API de Twitter. 

Grattage par lots
En raison du nombre limité d'appels d'API que l'on peut effectuer à l'aide d'un compte développeur basique et gratuit, (~ 900 appels toutes les 15 minutes avant que votre accès ne soit refusé), j'ai créé une fonction qui extrait 2500 tweets par exécution une fois toutes les 15 minutes


En raison du nombre limité d'appels d'API que l'on peut effectuer à l'aide d'un compte développeur basique et gratuit, (~ 900 appels toutes les 15 minutes avant que votre accès ne soit refusé), j'ai créé une fonction qui extrait 2500 tweets par exécution une fois toutes les 15 minutes extrait 3,00 et plus, mais cela m'a été refusé après le deuxième lot). Dans cette fonction, vous spécifiez:
paramètre de recherche tel que les mots clés et les hashtags, etc.
date de début, après laquelle tous les tweets seraient extraits (vous ne pouvez extraire que les tweets qui ne sont pas plus anciens que les 7 derniers jours)
nombre de tweets à tirer par exécution
nombre d'exécutions toutes les 15 minutes
J'ai extrait uniquement les métadonnées que j'ai jugées pertinentes pour mon cas. Vous pouvez explorer la liste des métadonnées de l'objet tweepy.Cursor en détail (c'est la vraie partie désordonnée).

### On obtient les informations suivantes :

        # user.screen_name - identifiant Twitter
        # user.description - description du compte
        # user.location - d'où vient-il tweeter
        # user.friends_count - non. des autres utilisateurs que l'utilisateur suit (suivant)
        # user.followers_count - non. des autres utilisateurs qui suivent cet utilisateur (suiveurs)
        # user.statuses_count - total des tweets par utilisateur
        # user.created_at - lors de la création du compte utilisateur
        # created_at - quand le tweet a été créé
        # retweet_count - non. de retweets
        # (obsolète) user.favourites_count - probablement aucun total. de tweets favoris par l'utilisateur
        # retweeted_status.full_text - texte intégral du tweet
        # tweet.entities ['hashtags'] - hashtags dans le tweet

In [1]:
# Import the libraries
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
import os
import time

In [2]:
# Clés de votre application
consumer_key = "dO73ZQhKgX2mA9qAER6cVmVDT"
consumer_secret = "YVjmIYiXpqRpRupVGCP4p6QzbA08lU4ihnhombPsn89KEM8rh4"

# le access_token est le token de l'application twitter que nous avons créée précédement
access_token = "1187541216-lgbnWetMfPJNP8BgiDOiBGwRNJCvLBGcuHfuurg"
access_token_secret = "Y2r7pWLWMgvQgSh0EmdD0Imts4lPMLvQYq1SDYvFZuJ2H"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [3]:
## Automating Scraping
# Calls API every 15 minutes to prevent overcalling

# 1. define a for-loop
# 2. define search parameter
# 3. define date period
# 4. define no. of tweets to pull

def collecttweets(search_words, date_since, numTweets, numRuns,language):

    ## Arguments:
    # search_words -> define a string of keywords for this function to extract
    # date_since -> define a date from which to start extracting the tweets 
    # numTweets -> number of tweets to extract per run
    # numRun -> number of runs to perform in this program - API calls are limited to once every 15 mins, so each run will be 15 mins apart.
    ##
    
    # dataframe pandas pour stocker
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags']
                                )
    # Define a for-loop to generate tweets at regular intervals
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        start_run2=time.asctime( time.localtime(time.time()) )
        
        
        # Collectez les tweets à l'aide de l'objet Cursor 
        # .Cursor () renvoie un objet que vous pouvez parcourir ou boucler pour accéder aux données collectées. 
        # Chaque élément de l'itérateur a divers attributs auxquels vous pouvez accéder pour obtenir des informations sur chaque tweet
        
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        tweets = tweepy.Cursor(api.search, q=search_words, lang=language, since=date_since, tweet_mode='extended').items(numTweets)

        # Stockez ces tweets dans une liste python 
        # Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]
        
        # Obtenez les informations suivantes (méthodes pour les appeler): 
        # user.screen_name - pseudo twitter 
        # user.description - description du compte 
        # user.location - où tweete-t-il depuis 
        # user.friends_count - no. des autres utilisateurs que l'utilisateur suit (suivant) 
        # user.followers_count - no. des autres utilisateurs qui suivent cet utilisateur (followers) 
        # user.statuses_count - total des tweets par l'utilisateur 
        # user.created_at - lorsque le compte utilisateur a été créé 
        # created_at - lorsque le tweet a été créé 
        # retweet_count - no. de retweets 
        # (obsolète) user.favourites_count - probablement aucun total. de tweets favoris par l'utilisateur
        # retweeted_status.full_text - texte intégral du tweet 
        # tweet.entities ['hashtags'] - hashtags dans le tweet
        
        # Commencez à gratter les tweets individuellement: 

        # Obtain the following info (methods to call them out):
            # user.screen_name - twitter handle
            # user.description - description of account
            # user.location - where is he tweeting from
            # user.friends_count - no. of other users that user is following (following)
            # user.followers_count - no. of other users who are following this user (followers)
            # user.statuses_count - total tweets by user
            # user.created_at - when the user account was created
            # created_at - when the tweet was created
            # retweet_count - no. of retweets
            # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
            # retweeted_status.full_text - full text of the tweet
            # tweet.entities['hashtags'] - hashtags in the tweet

        # Begin scraping the tweets individually:
        noTweets = 0

        for tweet in tweet_list:

            # Pull the values
            username = tweet.user.screen_name
            acctdesc = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']

            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                text = tweet.full_text

            # Add the 11 variables to the empty list - ith_tweet:
            ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]

            # Append to dataframe - db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet

            # increase counter - noTweets  
            noTweets += 1
        
        # Run ended:
        end_run = time.time()
        end_run2=time.asctime( time.localtime(time.time()) )
        duration_run = round((end_run-start_run)/60, 2)
        print('no. of tweets scraped for run {} is {}'.format(i, noTweets))
        print('no. of tweets scraped for run {} start {}'.format(i, start_run2))
        print('no. of tweets scraped for run {} end {}'.format(i, end_run2))                                                      
        print('time take for {} run to complete is {} minutes.'.format(i, duration_run))
        
        
        time.sleep(900) #15 minute sleep time

        
    # Once all runs have completed, save them to a single csv file:    
    # Obtain timestamp in a readable format:
    from datetime import datetime
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')

    # Define working path and filename
    path = os.getcwd()
    filename = path + '/' + to_csv_timestamp +'_'+'covid_tweets.csv'

    # Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)
    
    print('Scraping has completed!')

In [4]:
# Initialise these variables: 
search_words = "#Covid_19 OR #vaccincovid OR #vaccination OR #vaccincovid"
date_since = "2021-01-01"
numTweets = 1500
numRuns = 2 
language="fr"
# Call the function scraptweets
collecttweets(search_words, date_since, numTweets, numRuns,language)

no. of tweets scraped for run 0 is 1500
no. of tweets scraped for run 0 start Wed Jan 27 16:14:40 2021
no. of tweets scraped for run 0 end Wed Jan 27 16:15:47 2021
time take for 0 run to complete is 1.11 minutes.
no. of tweets scraped for run 1 is 1500
no. of tweets scraped for run 1 start Wed Jan 27 16:30:47 2021
no. of tweets scraped for run 1 end Wed Jan 27 16:31:47 2021
time take for 1 run to complete is 1.0 minutes.
Scraping has completed!
